<img src=./logo.png><br>
* ref: https://www.kaggle.com/c/titanic

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #플롯 그리는 라이브러리

# import warnings
# warnings.filterwarnings(action='ignore')
# 하면 경고 메세지 사라진다.

# 1. Data Load

In [2]:
train=pd.read_csv('./dataset/train.csv', sep=',')
print(train.shape) # **
print(train.info())
train.head()

(891, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# 2. EDA: Explorer Data Analasis (사전 데이터 탐색)

## Servived : Target Feature
-> Feature: 컬럼

In [129]:
print(train["Survived"].value_counts()) 

0    549
1    342
Name: Survived, dtype: int64


## 레코드수==유니크수

In [130]:
# check unique values
# print( len( pd.unique(train['Age']) ) ) # cf. unique function considers NaN as a value.
train.nunique()

PassengerId    891
Survived         2
Pclass           3
Name           891
Sex              2
Age             88
SibSp            7
Parch            7
Ticket         681
Fare           248
Cabin          147
Embarked         3
dtype: int64

In [131]:
# 결측 확인
train.isna().sum()[train.isna().sum()>0]

Age         177
Cabin       687
Embarked      2
dtype: int64

In [132]:
# select columns that nunique is 650 and over -> 너무 유니크한 컬럼 체크
train.nunique()[train.nunique()>=650]
train.nunique()[train.nunique()>=650].index.tolist()

# cf
# uni_col=[col for col in train.columns if train[col].nunique()>=650] # train.columns -> return series.(in에 array-like면 들어갈 수 있다.)
# print(uni_col)

['PassengerId', 'Name', 'Ticket']

In [133]:
# 티켓 drop 전 티켓에 의미 있는 정보 있는지 확인하기 위해 관계 파악 (1)
train[['Ticket','Survived']].value_counts().sort_values(ascending=False)

Ticket             Survived
CA. 2343           0           7
347082             0           7
347088             0           6
CA 2144            0           6
3101295            0           6
                              ..
STON/O 2. 3101280  0           1
PC 17604           1           1
PC 17603           1           1
A.5. 11206         0           1
WE/P 5735          1           1
Length: 730, dtype: int64

In [134]:
# 티켓 drop 전 티켓에 의미 있는 정보 있는지 확인하기 위해 관계 파악 (2)
train[['Ticket','Embarked',]].value_counts().sort_values(ascending=False)

# --> ticket에 의미 있는 정보 있다고 보기 어려우므로 drop

Ticket            Embarked
CA. 2343          S           7
347082            S           7
1601              S           7
347088            S           6
3101295           S           6
                             ..
SOTON/O2 3101287  S           1
SW/PP 751         S           1
STON/O2. 3101283  S           1
STON/O2. 3101282  S           1
347466            S           1
Length: 682, dtype: int64

In [135]:
# drop unique columns

uni_col=train.nunique()[train.nunique()>=650].index.tolist()
train=train.drop(uni_col, axis=1).copy()

train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S


# 피쳐 가공(Feature Engineering & Preprocessing)
-> EDA랑 엄격하게 나눌 수는 없다. (분석가마다 성향 다른데 동시에 하기도 하고 EDA를 먼저 하기도 하고)  
-> 결측처리도 엄밀히는 피쳐가공

## 결측 처리
* 이번엔 편의상 EDA하면서 바로 결측 처리
* Cabin  -> 함 살펴보자
* Age -> 호칭 별 평균 나이로 채우자
* Embarked -> 최빈값인 S로 채우자

### Embarked : 최빈도 채우기

In [136]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [137]:
# Embaked -> 최빈값 측정
train['Embarked'].mode().values    # return array
train['Embarked'].mode().values[0] # select specific value

train['Embarked']=train['Embarked'].fillna(train['Embarked'].mode().values[0])
train['Embarked'].isna().sum() # check nan value

0

In [138]:
# Survived와 Embarked의 관계 파악
pd.crosstab(train['Embarked'],train['Survived'])

# --> 관계 있다고 보기 어려움

Survived,0,1
Embarked,,
C,75,93
Q,47,30
S,427,219


### Age : Name 호칭 사용해 그들의 평균 나이로 채우기
-> 제대로 하려면 정규식 배워야 됨. 일단은 평균 나이 채워보자

In [139]:
avg_age = int(train['Age'].mean()) # int 처리 해서 소수점 날리기 / int는 값 캐스팅
train['Age']=train['Age'].fillna(avg_age).astype('int')

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    int32  
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     204 non-null    object 
 8   Embarked  891 non-null    object 
dtypes: float64(1), int32(1), int64(4), object(3)
memory usage: 59.3+ KB


### Cabin -> 점수 보면서 삭제 여부 결정
* Pclass=1인 승객 대부분이 cabin 가지고 있다.
* 삭제여부 미정 : Pclass 하나만 봐도 된다.
* A~T 세분화 되어있어서 C1, C2, C3 로 나눠도 되고

In [140]:
pd.unique(train['Cabin'])

# cabin과 살았니 죽었니 관계
train[['Cabin','Survived']].value_counts()

# 앞글자 하나만 딴 Cabin (Series에서 values 따로 안 떼어내도 됨.)

train['Cabin'].str[0:1] # 이걸 바로 Survived랑 비교할 수는 없음. -> 데이터프레임에 넣어줘야됨.
train['Cabin']=train['Cabin'].str[0:1]

In [141]:
# crosstab 이용해 Cabin(앞글자)와 Survived와의 관계 비교
pd.crosstab(train['Cabin'], train['Survived'])
# -->생존과는 뭔가 관계 없어보임.

Survived,0,1
Cabin,,
A,8,7
B,12,35
C,24,35
D,8,25
E,8,24
F,5,8
G,2,2
T,1,0


In [142]:
# crosstab 이용해 Cabin(앞글자)와 Pclass와의 관계 비교
pd.crosstab(train['Cabin'], train['Pclass'])

# --> 거의 관계 있어보임. -> 뺄까 말까? 어차피 Pclass봐도 될 거 같기도 하고 아직 판단이 안 선다.
# Pclass=1인 승객 대부분이 Cabin을 가지고 있다. -> 삭제 여부 미정.

Pclass,1,2,3
Cabin,,,
A,15,0,0
B,47,0,0
C,59,0,0
D,29,4,0
E,25,4,3
F,0,8,5
G,0,0,4
T,1,0,0


In [143]:
# cabin 결측: 공백 처리 ''
train['Cabin']=train['Cabin'].fillna('')
train['Cabin']

0       
1      C
2       
3      C
4       
      ..
886     
887    B
888     
889    C
890     
Name: Cabin, Length: 891, dtype: object

## Object 처리

<pre>
* cabin
* Sex
* Embarked
</pre>

In [144]:
# describe : object columns에 대해(별도 지정 필요)
train[['Cabin','Sex','Embarked']].describe() # top: 빈도. unique: nunique count: 개수

,Cabin,Sex,Embarked
count,891,891,891
unique,9,2,3
top,,male,S
freq,687,577,646


In [145]:
# 한 번에 object 타입인 컬럼 꺼내오기
col_list=train.select_dtypes(include='object').columns.tolist() # Return -> Subset DataFrame

# for문으로 unique 가져오기
for col in col_list:
    print(pd.unique(train[col]))


['male' 'female']
['' 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']
['S' 'C' 'Q']


In [146]:
# Sex : object -> int
sex_dict = {'male':1, 'female':0}
train['Sex2']=train['Sex'].map(sex_dict)
train['Sex2']

0      1
1      0
2      0
3      0
4      1
      ..
886    1
887    0
888    0
889    1
890    1
Name: Sex2, Length: 891, dtype: int64

In [147]:
# Sex : object -> int (get dummies 버전 : 후에 다중공선성 문제 확인용)
sex_dummy=pd.get_dummies(train['Sex'],prefix='Sex_')
train=pd.concat([train,sex_dummy],axis=1)
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Sex2,Sex__female,Sex__male
0,0,3,male,22,1,0,7.2500,,S,1,0,1
1,1,1,female,38,1,0,71.2833,C,C,0,1,0
2,1,3,female,26,0,0,7.9250,,S,0,1,0
3,1,1,female,35,1,0,53.1000,C,S,0,1,0
4,0,3,male,35,0,0,8.0500,,S,1,0,1


In [148]:
# Embarked : object -> int
train['Embarked'].replace(['S','C','Q'],[1,2,3], inplace=True)

In [149]:
# Cabin : object -> int
train['Cabin']=train['Cabin'].astype('category').cat.codes 

In [150]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Survived     891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          891 non-null    int32  
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
 7   Cabin        891 non-null    int8   
 8   Embarked     891 non-null    int64  
 9   Sex2         891 non-null    int64  
 10  Sex__female  891 non-null    uint8  
 11  Sex__male    891 non-null    uint8  
dtypes: float64(1), int32(1), int64(6), int8(1), object(1), uint8(2)
memory usage: 61.9+ KB


## 피쳐 가공: 불필요한 컬럼 모두 삭제

In [162]:
# train.info()

train=train.drop('Sex',axis=1)
train.info()
train.head()

KeyError: "['Sex'] not found in axis"

# ML(sklearn)

## 문제지 답안지 분리

In [163]:
# train 데이터와 test 데이터를 나눠보자
from sklearn.model_selection import train_test_split

In [164]:
# target data(y)와 datas(X) 분리

y=train['Survived']
X=train.drop(['Survived'], axis=1) # 간단

print(y)
print(X)
train.head()

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64
     Pclass  Age  SibSp  Parch     Fare  Cabin  Embarked  Sex2  Sex__female  \
0         3   22      1      0   7.2500      0         1     1            0   
1         1   38      1      0  71.2833      3         2     0            1   
2         3   26      0      0   7.9250      0         1     0            1   
3         1   35      1      0  53.1000      3         1     0            1   
4         3   35      0      0   8.0500      0         1     1            0   
..      ...  ...    ...    ...      ...    ...       ...   ...          ...   
886       2   27      0      0  13.0000      0         1     1            0   
887       1   19      0      0  30.0000      2         1     0            1   
888       3   29      1      2  23.4500      0         1     0            1   
889       1   26      0      0  30.0000      3         2     1       

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,Sex2,Sex__female,Sex__male
0,0,3,22,1,0,7.2500,0,1,1,0,1
1,1,1,38,1,0,71.2833,3,2,0,1,0
2,1,3,26,0,0,7.9250,0,1,0,1,0
3,1,1,35,1,0,53.1000,3,1,0,1,0
4,0,3,35,0,0,8.0500,0,1,1,0,1


In [165]:
# train set과 test set 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1024, shuffle=True)

In [166]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 10) (179, 10) (712,) (179,)


In [167]:
X.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin,Embarked,Sex2,Sex__female,Sex__male
0,3,22,1,0,7.2500,0,1,1,0,1
1,1,38,1,0,71.2833,3,2,0,1,0
2,3,26,0,0,7.9250,0,1,0,1,0
3,1,35,1,0,53.1000,3,1,0,1,0
4,3,35,0,0,8.0500,0,1,1,0,1


In [168]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

## 모델 선정 & 학습 & 예측 &  평가

In [169]:
# import model

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score

In [171]:
model_list=[RandomForestClassifier(n_estimators=300, random_state=1024),
            LogisticRegression(penalty="l2"),
            KNeighborsClassifier(),
            DecisionTreeClassifier(),
            LinearSVC()
           ]

for model in model_list:
    model.fit(X_train, y_train)
    answer=model.predict(X_test)
    acc_score=accuracy_score(y_test, answer) 
    print(f"{model.__class__.__name__} \t\t\t 정확도 : {acc_score:4f}") 

RandomForestClassifier 			 정확도 : 0.810056
LogisticRegression 			 정확도 : 0.804469
KNeighborsClassifier 			 정확도 : 0.720670
DecisionTreeClassifier 			 정확도 : 0.804469
LinearSVC 			 정확도 : 0.765363


c:\ai\pythonproject\venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\ai\pythonproject\venv\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# 모델 튜닝 & 검증